In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Generate synthetic data
np.random.seed(42)

def generate_data(num_samples=1000):
    x = np.random.uniform(-2, 2, num_samples)
    y = np.random.uniform(-2, 2, num_samples)
    z = np.random.uniform(-2, 2, num_samples)
    f_xyz = np.sin(x) + np.cos(y) * np.power(z, 2)
    features = np.vstack((x, y, z)).T
    outputs = f_xyz
    return features, outputs

features, outputs = generate_data()

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, outputs, test_size=0.2, random_state=42)

# Convert to TensorFlow tensors
X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test_tf = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tf = tf.convert_to_tensor(y_test, dtype=tf.float32)

# Neural network parameters
input_size = 3
hidden_size = 64
output_size = 1

# Weights and biases
W1 = tf.Variable(tf.random.normal([input_size, hidden_size], stddev=0.01), name='W1')
b1 = tf.Variable(tf.zeros([hidden_size]), name='b1')
W2 = tf.Variable(tf.random.normal([hidden_size, hidden_size], stddev=0.01), name='W2')
b2 = tf.Variable(tf.zeros([hidden_size]), name='b2')
W3 = tf.Variable(tf.random.normal([hidden_size, output_size], stddev=0.01), name='W3')
b3 = tf.Variable(tf.zeros([output_size]), name='b3')

def forward_pass(X):
    """
    Performs the forward pass of the neural network.
    """
    Z1 = tf.add(tf.matmul(X, W1), b1)
    A1 = tf.nn.relu(Z1)
    Z2 = tf.add(tf.matmul(A1, W2), b2)
    A2 = tf.nn.relu(Z2)
    Z3 = tf.add(tf.matmul(A2, W3), b3)
    return Z3

def compute_loss(y_true, y_pred):
    """
    Computes the Mean Squared Error.
    """
    return tf.reduce_mean(tf.square(y_true - y_pred))


In [ ]:
optimizer = tf.optimizers.Adam(learning_rate=0.001)

@tf.function
def train_step(X, y):
    with tf.GradientTape() as tape:
        y_pred = forward_pass(X)
        loss = compute_loss(y, y_pred)
    gradients = tape.gradient(loss, [W1, b1, W2, b2, W3, b3])
    optimizer.apply_gradients(zip(gradients, [W1, b1, W2, b2, W3, b3]))
    return loss


In [ ]:
# Training parameters
num_epochs = 100
batch_size = 32
num_batches = int(X_train_tf.shape[0] / batch_size)

# Convert training data to TensorFlow Dataset for easy batching
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tf, y_train_tf))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Training loop
for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()

    for X_batch, y_batch in train_dataset:
        loss = train_step(X_batch, y_batch)
        epoch_loss_avg.update_state(loss)

    # Print epoch loss
    print(f"Epoch {epoch + 1}: Loss = {epoch_loss_avg.result().numpy()}")

# Evaluate the model
y_pred = forward_pass(X_test_tf)
test_loss = compute_loss(y_test_tf, y_pred)
print(f"Test Loss: {test_loss.numpy()}")


Epoch 1: Loss = 2.006441116333008
Epoch 2: Loss = 1.8089503049850464
Epoch 3: Loss = 1.6864248514175415
Epoch 4: Loss = 1.6811546087265015
Epoch 5: Loss = 1.6835821866989136
Epoch 6: Loss = 1.681503415107727
Epoch 7: Loss = 1.6818797588348389
Epoch 8: Loss = 1.6809359788894653
Epoch 9: Loss = 1.6829965114593506
Epoch 10: Loss = 1.6815180778503418
Epoch 11: Loss = 1.6833759546279907
Epoch 12: Loss = 1.6844717264175415
Epoch 13: Loss = 1.6818554401397705
Epoch 14: Loss = 1.681178331375122
Epoch 15: Loss = 1.6794716119766235
Epoch 16: Loss = 1.6804394721984863
Epoch 17: Loss = 1.677107334136963
Epoch 18: Loss = 1.679297924041748
Epoch 19: Loss = 1.6771857738494873
Epoch 20: Loss = 1.6805959939956665
Epoch 21: Loss = 1.6833518743515015
Epoch 22: Loss = 1.6808096170425415
Epoch 23: Loss = 1.6762219667434692
Epoch 24: Loss = 1.6780892610549927
Epoch 25: Loss = 1.679823398590088
Epoch 26: Loss = 1.6820461750030518
Epoch 27: Loss = 1.6796519756317139
Epoch 28: Loss = 1.6797932386398315
Epoch 2